In [1]:
import sys
sys.path.append('/home/robson/curso_spark_pyspark_udemy/utils')

from spark_session import get_spark_session

spark = get_spark_session("Particionamento")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/05 10:50:24 WARN Utils: Your hostname, LAPTOP-CO0U45G1, resolves to a loopback address: 127.0.1.1; using 172.20.235.103 instead (on interface eth0)
25/08/05 10:50:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/05 10:50:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/05 10:50:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/05 10:50:39 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
spark.sql("show databases").show()

25/08/05 10:51:05 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/08/05 10:51:05 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore robson@127.0.1.1


+---------+
|namespace|
+---------+
|  default|
|     desp|
+---------+



In [3]:
spark.sql('use desp').show()

++
||
++
++



In [6]:
caminho_churn = '/home/robson/curso_spark_pyspark_udemy/Churn.csv'
churn = spark.read.csv(caminho_churn, header=True, inferSchema=True, sep=';')
churn.show()

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
|        645|    Spain|  Male| 44|     8|11375578|            2|        1|             0|       

In [7]:
churn.write.partitionBy('Geography').saveAsTable('churn_geo')

25/08/05 11:09:52 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
25/08/05 11:09:53 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist


In [8]:
spark.sql('select * from churn_geo').show()

+-----------+------+---+------+--------+-------------+---------+--------------+---------------+------+---------+
|CreditScore|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|Geography|
+-----------+------+---+------+--------+-------------+---------+--------------+---------------+------+---------+
|        619|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|   France|
|        502|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|   France|
|        699|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|   France|
|        822|  Male| 50|     7|       0|            2|        1|             1|         100628|     0|   France|
|        501|  Male| 44|     4|14205107|            2|        0|             1|         749405|     0|   France|
|        684|  Male| 27|     2|13460388|            1|        1|             1|        7172573| 

In [9]:
spark.sql('select * from churn_geo where Geography = "Spain"').show()

+-----------+------+---+------+--------+-------------+---------+--------------+---------------+------+---------+
|CreditScore|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|Geography|
+-----------+------+---+------+--------+-------------+---------+--------------+---------------+------+---------+
|        608|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|    Spain|
|        850|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|    Spain|
|        645|  Male| 44|     8|11375578|            2|        1|             0|       14975671|     1|    Spain|
|        497|  Male| 24|     3|       0|            2|        1|             0|        7639001|     0|    Spain|
|        635|Female| 35|     7|       0|            2|        1|             1|        6595165|     0|    Spain|
|        549|Female| 24|     9|       0|            2|        1|             1|        1440641| 

In [10]:
churn.write.bucketBy(3, 'Geography').saveAsTable('churn_geo2')

In [11]:
spark.sql('select * from churn_geo2').show()

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
|        645|    Spain|  Male| 44|     8|11375578|            2|        1|             0|       